In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from functools import reduce

## Datos Gobiernos Locales

In [2]:
 # carga de gobienos locales 
df_goblocales = gpd.read_file("../GobiernosLocales/GobiernosLocales.shp")
df_goblocales.rename(columns={'Provincia':'provincia', 'Departamen':'departamen', 'Municipio':'municipio', 'UTA2020':'uta2020'}, inplace=True)

df_goblocales = df_goblocales.to_crs("EPSG:3035")
df_goblocales['area_total_k2'] = df_goblocales.area / 1000000
df_goblocales = df_goblocales.to_crs("EPSG:4326")

In [3]:
# carga de las capas relacionadas a hidrografía: cuencas, ríos, espejos
df_cuencas = gpd.read_file("../CapasGeograficasOriginales/CuencasHidrograficas.geojson")
df_rios_perenne = gpd.read_file("../CapasGeograficasOriginales/CursoDeAguaPerenne.geojson")
df_rios_interm = gpd.read_file("../CapasGeograficasOriginales/CursoDeAguaIntermitente.geojson")
df_rios_sup = gpd.read_file("../CapasGeograficasOriginales/CursoDeAgua_Grandes.geojson")
df_espejos_perenne = gpd.read_file("../CapasGeograficasOriginales/EspejoAguaPerenne_simple.geojson")
df_espejos_interm = gpd.read_file("../CapasGeograficasOriginales/EspejoAguaIntermitente_simple.geojson")

In [4]:
# carga de la capa de áreas protegidas
df_areaprotegida = gpd.read_file("../CapasGeograficasOriginales/AreasProtegidas.geojson") 

In [5]:
# carga de las capas relacionadas al tratamiento de la basura
df_basurales = gpd.read_file("../CapasGeograficasOriginales/VertederosBasureros.geojson") 
df_rellenos = gpd.read_file("../CapasGeograficasOriginales/RellenoSanitario.geojson") 
df_plseparacion = gpd.read_file("../CapasGeograficasOriginales/PlantaSeparacion.geojson") 
df_plresiduos = gpd.read_file("../CapasGeograficasOriginales/PlantaTratamientoResiduos_Pto.geojson") 
df_plefluentes = gpd.read_file("../CapasGeograficasOriginales/PlantaTratamientoEfluentes_Pto.geojson") 
df_plpotabil = gpd.read_file("../CapasGeograficasOriginales/PlantaPotabilizadoraDeAgua_Pto.geojson") 

## Hidrografía

### Cuencas Hidrográficas

In [6]:
# cruce de las cuencas con los gobiernos locales y exrpotación del resultado
df_inter_gobcuencas = gpd.overlay(df_goblocales, df_cuencas, how='intersection')
df_inter_gobcuencas.to_file("../CapasGeograficasCruzadas/Cuencas_GobLocal.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# Generación de los valores de las distintas variable
# Cantidad de cuencas
df_cuencas_cant = pd.DataFrame(df_inter_gobcuencas.uta2020.value_counts()).reset_index()
df_cuencas_cant.columns = ['uta2020', 'cuenca_cant']
# Nombre de las cuencas
df_inter_gobcuencas_nombres = df_inter_gobcuencas.groupby(['uta2020'])['NOMBRE'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()

In [8]:
# unipon de las capas de ríos y espejos, y disolución de la capa
df_agua = pd.concat([df_rios_sup, df_espejos_perenne, df_espejos_interm])
df_agua['dis'] = 'X'
df_agua = gpd.GeoDataFrame(df_agua, geometry='geometry')
df_agua.dissolve(by='dis')
# cruce de la capa con gobiernos locales
df_inter_gobagua = gpd.overlay(df_goblocales, df_agua[['geometry']], how='intersection')
# cálculo de superficie
df_inter_gobagua = df_inter_gobagua.to_crs("EPSG:3035")
df_inter_gobagua['sup_agua'] = df_inter_gobagua.area / 1000000
df_inter_gobagua = df_inter_gobagua.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gobagua.to_file("../CapasGeograficasCruzadas/aguaArea_goblocal.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [9]:
# calculo de la superfie con agua y porcentaje sobre el total del gobieno local
df_agua_sup = pd.DataFrame(df_inter_gobagua.groupby(['uta2020'])['sup_agua'].sum()).reset_index()
df_agua_sup = df_agua_sup.merge(df_goblocales[["uta2020", 'area_total_k2']], how='left', on="uta2020")
df_agua_sup.sup_agua.fillna(0, inplace=True)
df_agua_sup['sup_agua_porc'] = df_agua_sup['sup_agua'] / (df_agua_sup['area_total_k2'] * 1000000)
df_agua_sup = df_agua_sup[["uta2020", 'sup_agua', 'sup_agua_porc']]

In [10]:
# generación del dataframe de cuencas
dfs_cuencas = [df_cuencas_cant, df_inter_gobcuencas_nombres, df_agua_sup]
dfs_cuencas_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_cuencas)
# cambio del nombre de las columnas
dfs_cuencas_merged.rename(columns={'cuenca_cant':'u_cue_tot', 'NOMBRE':'st_cue_nam', 'sup_agua':'k2_cue_sag', 'sup_agua_porc':'pc_cue_sag'}, inplace=True)
# formateo de los campos numéricos
dfs_cuencas_merged.k2_cue_sag = round(dfs_cuencas_merged.k2_cue_sag, 2)
dfs_cuencas_merged.pc_cue_sag = dfs_cuencas_merged.pc_cue_sag * 100
dfs_cuencas_merged.pc_cue_sag = round(dfs_cuencas_merged.pc_cue_sag, 2)
# extracción de los nombres de los campos
df_columns_cuencas = list(dfs_cuencas_merged.columns)

In [11]:
# cruce de los datos generados con los gobienros locales
df_goblocales_cuencas_cant = df_goblocales.merge(dfs_cuencas_merged, how='left', on="uta2020")
df_goblocales_cuencas_cant.u_cue_tot.fillna(0, inplace=True)
df_goblocales_cuencas_cant.k2_cue_sag.fillna(0, inplace=True)
df_goblocales_cuencas_cant.pc_cue_sag.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_cuencas_cant = df_goblocales_cuencas_cant.drop(columns=['area_total_k2'])
df_goblocales_cuencas_cant.loc[df_goblocales_cuencas_cant['buffer'] == 1, 'st_cue_nam'] = None
df_goblocales_cuencas_cant.loc[df_goblocales_cuencas_cant['buffer'] == 1, ['u_cue_tot', 'k2_cue_sag', 'pc_cue_sag']] = float("NaN")
# exportación de los datos generados a csv y shp
df_goblocales_cuencas_cant[df_columns_cuencas].to_csv("../BD_Procesada/MedioAmbienteCuencas_GobiernosLocales.csv", encoding='utf-8')
schema_cuenca_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 
                    'tipgobloc':'str', 'uta2020':'str',
                    'u_cue_tot':'int', 'st_cue_nam':'str', 'k2_cue_sag':'float:10.2', 'pc_cue_sag':'float:10.2'}}
df_goblocales_cuencas_cant[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_cuencas+['geometry']].to_file(
    "../CapasGeograficasProcesadas/MedioAmbienteCuencas_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_cuenca_dic,
    encoding='utf-8')
df_goblocales_cuencas_cant.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,Fdc,Fuente,Latitud,Longitud,buffer,geometry,u_cue_tot,st_cue_nam,k2_cue_sag,pc_cue_sag
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,None,Capa Departamentos IGN,-34.60642188555109,-58.37153965302685,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",1.0,['Desagüe al Río de la Plata al Sur de Río Sam...,1.02,0.0
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,None,Capa Departamentos IGN,-34.58576499379653,-58.39489181181946,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",1.0,['Desagüe al Río de la Plata al Sur de Río Sam...,0.03,0.0
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,None,Capa Departamentos IGN,-34.61384308168632,-58.40265145122171,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",1.0,['Desagüe al Río de la Plata al Sur de Río Sam...,0.00,0.0
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,None,Capa Departamentos IGN,-34.64207942578216,-58.38745506898847,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",1.0,['Desagüe al Río de la Plata al Sur de Río Sam...,1.15,0.0
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,None,Capa Departamentos IGN,-34.61736992378499,-58.42057219028574,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",1.0,['Desagüe al Río de la Plata al Sur de Río Sam...,0.00,0.0


### Cursos de Agua

In [12]:
# cruce de los ríos perenne con la capa de gob locales
df_inter_gob_rios_perenne = gpd.overlay(df_rios_perenne, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_rios_perenne = df_inter_gob_rios_perenne.loc[df_inter_gob_rios_perenne.uta2020.notnull()].reset_index()
# cálculo de la longitud de los tramos de ríos
df_inter_gob_rios_perenne = df_inter_gob_rios_perenne.to_crs("EPSG:3035")
df_inter_gob_rios_perenne['long_km'] = df_inter_gob_rios_perenne.length / 1000
df_inter_gob_rios_perenne = df_inter_gob_rios_perenne.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gob_rios_perenne.to_file("../CapasGeograficasCruzadas/cursosPerennes_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2361 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [13]:
# calculo de las variables
df_inter_gob_rios_perenne.fna = df_inter_gob_rios_perenne.fna.astype(str)
# cantidad de ríos
df_inter_gob_rios_perenne_cantidad = df_inter_gob_rios_perenne.groupby(['uta2020'])['fna'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_rios_perenne_cantidad.rename(columns = {'fna':'perenne_cant'}, inplace=True)
# nombre de los ríos
df_inter_gob_rios_perenne_nombres = df_inter_gob_rios_perenne.loc[(df_inter_gob_rios_perenne.fna.notnull()) & (df_inter_gob_rios_perenne.fna != '-1')& (df_inter_gob_rios_perenne.fna != '-2')& (df_inter_gob_rios_perenne.fna != 'nan')].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_rios_perenne_nombres.rename(columns = {'fna':'perenne_nombres'}, inplace=True)
# longitud de los ríos
df_inter_gob_rios_perenne_long = df_inter_gob_rios_perenne.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_rios_perenne_long.rename(columns = {'long_km':'perenne_long_km'}, inplace=True)

In [14]:
# cruce de los ríos intermitentes con la capa de gob locales
df_inter_gob_rios_interm = gpd.overlay(df_rios_interm, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_rios_interm = df_inter_gob_rios_interm.loc[df_inter_gob_rios_interm.uta2020.notnull()].reset_index()
# cálculo de la longitud de los tramos de ríos
df_inter_gob_rios_interm = df_inter_gob_rios_interm.to_crs("EPSG:3035")
df_inter_gob_rios_interm['long_km'] = df_inter_gob_rios_interm.length / 1000
df_inter_gob_rios_interm = df_inter_gob_rios_interm.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gob_rios_interm.to_file("../CapasGeograficasCruzadas/cursosIntermitentes_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [15]:
# calculo de las variables
df_inter_gob_rios_interm.fna = df_inter_gob_rios_interm.fna.astype(str)
# cantidad de ríos
df_inter_gob_rios_interm_cantidad = df_inter_gob_rios_interm.groupby(['uta2020'])['fna'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_rios_interm_cantidad.rename(columns = {'fna':'interm_cant'}, inplace=True)
# nombre de los ríos
df_inter_gob_rios_interm_nombres = df_inter_gob_rios_interm.loc[(df_inter_gob_rios_interm.fna.notnull()) & (df_inter_gob_rios_interm.fna != '-1')& 
(df_inter_gob_rios_interm.fna != '-2')& (df_inter_gob_rios_interm.fna != 'nan')].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_rios_interm_nombres.rename(columns = {'fna':'interm_nombres'}, inplace=True)
# longitud de los ríos
df_inter_gob_rios_interm_long = df_inter_gob_rios_interm.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_rios_interm_long.rename(columns = {'long_km':'interm_long_km'}, inplace=True)

In [16]:
# merge de todos los dataframes de cursos de agua
dfs_rios = [df_inter_gob_rios_perenne_cantidad, df_inter_gob_rios_interm_cantidad, df_inter_gob_rios_perenne_nombres, df_inter_gob_rios_interm_nombres, 
            df_inter_gob_rios_perenne_long, df_inter_gob_rios_interm_long]
dfs_rios_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_rios)

# limpieza del dataframe
dfs_rios_merged.perenne_cant.fillna(0, inplace=True)
dfs_rios_merged.interm_cant.fillna(0, inplace=True)
dfs_rios_merged['cant_total'] = dfs_rios_merged['perenne_cant'] + dfs_rios_merged['interm_cant']
dfs_rios_merged['perenne_bool'] = 1
dfs_rios_merged.loc[dfs_rios_merged.perenne_cant == 0, 'perenne_bool'] = 0
dfs_rios_merged['interm_bool'] = 1
dfs_rios_merged.loc[dfs_rios_merged.interm_cant == 0, 'interm_bool'] = 0

# nombres de columnas según estandarización
dfs_rios_merged.rename(columns={'perenne_cant':'u_cap_tot', 'interm_cant':'u_cai_tot', 'cant_total':'u_ca_tot', 'perenne_nombres':'st_cap_nam', 'interm_nombres':'st_cai_nam',
    'perenne_long_km':'km_cap_lon', 'interm_long_km':'km_cai_lon', 'perenne_bool':'bo_cap_tot', 'interm_bool':'bo_cai_tot'}, inplace=True)

# formateo de los campos numéricos
dfs_rios_merged.km_cap_lon = round(dfs_rios_merged.km_cap_lon, 3)
dfs_rios_merged.km_cai_lon = round(dfs_rios_merged.km_cai_lon, 3)

# extracción de los nombres de los campos
df_columns_cursos = list(dfs_rios_merged.columns)

In [17]:
# cruce de los datos generados con los gobienros locales
df_goblocales_cursos = df_goblocales.merge(dfs_rios_merged, how='left', on="uta2020")
df_goblocales_cursos.u_cap_tot.fillna(0, inplace=True)
df_goblocales_cursos.km_cap_lon.fillna(0, inplace=True)
df_goblocales_cursos.u_cai_tot.fillna(0, inplace=True)
df_goblocales_cursos.km_cai_lon.fillna(0, inplace=True)
df_goblocales_cursos.u_ca_tot.fillna(0, inplace=True)
df_goblocales_cursos.bo_cap_tot.fillna(0, inplace=True)
df_goblocales_cursos.bo_cai_tot.fillna(0, inplace=True)

# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_cursos = df_goblocales_cursos.drop(columns=['area_total_k2'])
df_goblocales_cursos.loc[df_goblocales_cursos['buffer'] == 1, ['st_cap_nam', 'st_cai_nam']] = None
df_goblocales_cursos.loc[df_goblocales_cursos['buffer'] == 1, ['u_cap_tot', 'u_cai_tot', 'km_cap_lon', 'km_cai_lon', 'u_ca_tot', 'bo_cap_tot', 'bo_cai_tot']] = float("NaN")

# exportación de los datos generados a csv y shp
df_goblocales_cursos[df_columns_cursos].to_csv("../BD_Procesada/MedioAmbienteCursosAgua_GobiernosLocales.csv", encoding='utf-8')
schema_cursos_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 
                    'tipgobloc':'str', 'uta2020':'str',
                    'u_cap_tot':'int', 'u_cai_tot':'int', 'st_cap_nam':'str', 'st_cai_nam':'str', 'km_cap_lon':'float:10.3', 
                    'km_cai_lon':'float:10.3', 'u_ca_tot':'int', 'bo_cap_tot':'int', 'bo_cai_tot':'int'}}
df_goblocales_cursos[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_cursos +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MedioAmbienteCursosAgua_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_cursos_dic,
    encoding='utf-8')
df_goblocales_cursos.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,geometry,u_cap_tot,u_cai_tot,st_cap_nam,st_cai_nam,km_cap_lon,km_cai_lon,u_ca_tot,bo_cap_tot,bo_cai_tot
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",0.0,0.0,NaN,NaN,0.000,0.0,0.0,0.0,0.0
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",0.0,0.0,NaN,NaN,0.000,0.0,0.0,0.0,0.0
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",0.0,0.0,NaN,NaN,0.000,0.0,0.0,0.0,0.0
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",1.0,0.0,['Riachuelo'],NaN,7.051,0.0,1.0,1.0,0.0
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",0.0,0.0,NaN,NaN,0.000,0.0,0.0,0.0,0.0


### Espejos de Agua

In [18]:
# cruce de los espejos de agua perenne con la capa de gob locales
df_inter_gob_espejos_perenne = gpd.overlay(df_espejos_perenne, df_goblocales, how='intersection')
df_inter_gob_espejos_perenne = df_inter_gob_espejos_perenne.loc[df_inter_gob_espejos_perenne.uta2020.notnull()].reset_index()
# cálculo del área de los espejos
df_inter_gob_espejos_perenne = df_inter_gob_espejos_perenne.to_crs("EPSG:3035")
df_inter_gob_espejos_perenne['area_k2'] = df_inter_gob_espejos_perenne.area / 1000000
df_inter_gob_espejos_perenne = df_inter_gob_espejos_perenne.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gob_espejos_perenne.to_file("../CapasGeograficasCruzadas/espejosPerennes_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [19]:
# calculo de las variables
df_inter_gob_espejos_perenne.fna = df_inter_gob_espejos_perenne.fna.astype(str)
# cantidad de espejos
df_inter_gob_espejos_perenne_cantidad = df_inter_gob_espejos_perenne.groupby(['uta2020'])['fna'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_espejos_perenne_cantidad.rename(columns = {'fna':'perenne_cant'}, inplace=True)
# lista de nombres de espejos
df_inter_gob_espejos_perenne_nombres = df_inter_gob_espejos_perenne.loc[(df_inter_gob_espejos_perenne.fna.notnull()) & (df_inter_gob_espejos_perenne.fna != '-1')& (df_inter_gob_espejos_perenne.fna != '-2')& (df_inter_gob_espejos_perenne.fna != 'nan')].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_espejos_perenne_nombres.rename(columns = {'fna':'perenne_nombres'}, inplace=True)
# superficie de espejos de agua
df_inter_gob_espejos_perenne_area = df_inter_gob_espejos_perenne.groupby(['uta2020'])['area_k2'].sum().reset_index()
df_inter_gob_espejos_perenne_area.rename(columns = {'area_k2':'perenne_area_k2'}, inplace=True)

In [20]:
# cruce de los espejos de agua intermitente con la capa de gob locales
df_inter_gob_espejos_interm = gpd.overlay(df_espejos_interm, df_goblocales, how='intersection')
df_inter_gob_espejos_interm = df_inter_gob_espejos_interm.loc[df_inter_gob_espejos_interm.uta2020.notnull()].reset_index()
# cálculo del área de los espejos
df_inter_gob_espejos_interm = df_inter_gob_espejos_interm.to_crs("EPSG:3035")
df_inter_gob_espejos_interm['area_k2'] = df_inter_gob_espejos_interm.area / 1000000
df_inter_gob_espejos_interm = df_inter_gob_espejos_interm.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gob_espejos_interm.to_file("../CapasGeograficasCruzadas/espejosInterm_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [21]:
# calculo de las variables
df_inter_gob_espejos_interm.fna = df_inter_gob_espejos_interm.fna.astype(str)
# cantidad de espejos
df_inter_gob_espejos_interm_cantidad = df_inter_gob_espejos_interm.groupby(['uta2020'])['fna'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_espejos_interm_cantidad.rename(columns = {'fna':'interm_cant'}, inplace=True)
# lista de nombres de espejos
df_inter_gob_espejos_interm_nombres = df_inter_gob_espejos_interm.loc[(df_inter_gob_espejos_interm.fna.notnull()) & (df_inter_gob_espejos_interm.fna != -1)
    & (df_inter_gob_espejos_interm.fna != -2)& (df_inter_gob_espejos_interm.fna != 'nan')].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_espejos_interm_nombres.rename(columns = {'fna':'interm_nombres'}, inplace=True)
# superficie de espejos de agua
df_inter_gob_espejos_interm_area = df_inter_gob_espejos_interm.groupby(['uta2020'])['area_k2'].sum().reset_index()
df_inter_gob_espejos_interm_area.rename(columns = {'area_k2':'interm_area_k2'}, inplace=True)

In [22]:
# merge de todos los dataframes de espejos de agua
dfs_espejos_perenne = [df_inter_gob_espejos_perenne_cantidad, df_inter_gob_espejos_interm_cantidad, df_inter_gob_espejos_perenne_nombres, df_inter_gob_espejos_interm_nombres,
                df_inter_gob_espejos_perenne_area, df_inter_gob_espejos_interm_area]
dfs_espejos_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_espejos_perenne)
# limpieza del dataframe
dfs_espejos_merged.perenne_cant.fillna(0, inplace=True)
dfs_espejos_merged.interm_cant.fillna(0, inplace=True)
dfs_espejos_merged['cant_total'] = dfs_espejos_merged['perenne_cant'] + dfs_espejos_merged['interm_cant'] 
dfs_espejos_merged['perenne_bool'] = 1
dfs_espejos_merged.loc[dfs_espejos_merged.perenne_cant == 0, 'perenne_bool'] = 0
dfs_espejos_merged['interm_bool'] = 1
dfs_espejos_merged.loc[dfs_espejos_merged.interm_cant == 0, 'interm_bool'] = 0
# nombres de columnas según estandarización
dfs_espejos_merged.rename(columns={'perenne_cant':'u_eap_tot', 'interm_cant':'u_eai_tot', 'cant_total':'u_ea_tot', 'perenne_nombres':'st_eap_nam', 'interm_nombres':'st_eai_nam',
    'perenne_area_k2':'k2_eap_sup', 'interm_area_k2':'k2_eai_sup', 'perenne_bool':'bo_eap_tot', 'interm_bool':'bo_eai_tot'}, inplace=True)
# formateo de los campos numéricos
dfs_espejos_merged.k2_eap_sup = round(dfs_espejos_merged.k2_eap_sup, 2)
dfs_espejos_merged.k2_eai_sup = round(dfs_espejos_merged.k2_eai_sup, 2)
# extracción de los nombres de los campos
df_columns_espejos = list(dfs_espejos_merged.columns)

In [23]:
# cruce de los datos generados con los gobienros locales
df_goblocales_espejos = df_goblocales.merge(dfs_espejos_merged, how='left', on="uta2020")
df_goblocales_espejos.u_eap_tot.fillna(0, inplace=True)
df_goblocales_espejos.k2_eap_sup.fillna(0, inplace=True)
df_goblocales_espejos.u_eai_tot.fillna(0, inplace=True)
df_goblocales_espejos.k2_eai_sup.fillna(0, inplace=True)
df_goblocales_espejos.u_ea_tot.fillna(0, inplace=True)
df_goblocales_espejos.bo_eap_tot.fillna(0, inplace=True)
df_goblocales_espejos.bo_eai_tot.fillna(0, inplace=True)

# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_espejos = df_goblocales_espejos.drop(columns=['area_total_k2'])
df_goblocales_espejos.loc[df_goblocales_espejos['buffer'] == 1, ['st_eap_nam', 'st_eai_nam']] = None
df_goblocales_espejos.loc[df_goblocales_espejos['buffer'] == 1, ['u_eap_tot', 'u_eai_tot', 'k2_eap_sup', 'k2_eai_sup', 'u_ea_tot', 'bo_eap_tot', 'bo_eai_tot']] = float("NaN")

# exportación de los datos generados a csv y shp
df_goblocales_espejos[df_columns_espejos].to_csv("../BD_Procesada/MedioAmbienteEspejosAgua_GobiernosLocales.csv", encoding='utf-8')
schema_espejos_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 
                    'tipgobloc':'str', 'uta2020':'str',
                    'u_eap_tot':'int', 'u_eai_tot':'int', 'st_eap_nam':'str', 'st_eai_nam':'str', 'k2_eap_sup':'float:10.2', 
                    'k2_eai_sup':'float:10.2', 'u_ea_tot':'int', 'bo_eap_tot':'int', 'bo_eai_tot':'int'}}
df_goblocales_espejos[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_espejos +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MedioAmbienteEspejosAgua_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_espejos_dic,
    encoding='utf-8')
df_goblocales_espejos.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,geometry,u_eap_tot,u_eai_tot,st_eap_nam,st_eai_nam,k2_eap_sup,k2_eai_sup,u_ea_tot,bo_eap_tot,bo_eai_tot
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",1.0,0.0,NaN,NaN,0.5,0.0,1.0,1.0,0.0
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Áreas Protegidas

In [24]:
# disolución de la capa 
df_areaprotegida['dissolvefield'] = 'X'
df_areaprotegida_dissolve = df_areaprotegida.dissolve(by="dissolvefield")
print(len(df_areaprotegida_dissolve))

# cruce de las áreas protegidas disueltas con la capa de gob locales
df_inter_gobareasdis = gpd.overlay(df_goblocales, df_areaprotegida_dissolve, how='intersection')

# cálculo de la superficie de las áreas protegidas
df_inter_gobareasdis = df_inter_gobareasdis.to_crs("EPSG:3035")
df_inter_gobareasdis['sup_aprot_k2'] = df_inter_gobareasdis.area / 1000000
df_inter_gobareasdis = df_inter_gobareasdis.to_crs("EPSG:4326")

1


c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `keep_geom_type=True` in overlay resulted in 2716 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  import sys


In [25]:
# calculo de las variables
# superficie cubierta por áreas protegidas por gob local
df_sup_areaprotegidas = pd.DataFrame(df_inter_gobareasdis.groupby("uta2020")["sup_aprot_k2"].sum()).reset_index()
df_goblocales_aprot = df_sup_areaprotegidas.merge(df_goblocales, how='left', on="uta2020")
df_goblocales_aprot["sup_aprot_k2"].fillna(0, inplace=True)
# porcentaje del área del gob local cubierta por área protegida
df_goblocales_aprot['porc_sup_prot'] = df_goblocales_aprot['sup_aprot_k2'] / df_goblocales_aprot['area_total_k2']
df_goblocales_aprot.loc[df_goblocales_aprot.porc_sup_prot > 1, "porc_sup_prot"] = 1
df_goblocales_aprot = df_goblocales_aprot[['uta2020','sup_aprot_k2','porc_sup_prot']]
df_goblocales_aprot.head()

,uta2020,sup_aprot_k2,porc_sup_prot
0,020072007,3.342025,1.868317e-01
1,020282028,0.000019,8.199852e-07
2,020912091,0.219054,1.464963e-02
3,060700070,8.593900,5.621259e-03
4,060910091,84.500326,3.805014e-01


#### Orden, Tipo, Cantidad y Nombre de reservas

In [26]:
# cruce de las áreas protegidas con la capa de gob locales
df_inter_gobareas = gpd.overlay(df_goblocales, df_areaprotegida, how='intersection')
# cálculo de la superficie de las áreas protegidas
df_inter_gobareas = df_inter_gobareas.to_crs("EPSG:3035")
df_inter_gobareas['sup_aprot_jap'] = df_inter_gobareas.area / 1000000
df_inter_gobareas = df_inter_gobareas.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gobareas.to_file("../CapasGeograficasCruzadas/areaProtegida_goblocal.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2802 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [27]:
# calculo de las variables
df_inter_gobareas.fna = df_inter_gobareas.fna.astype(str)
# cantidad de áreas protegidas
df_inter_gobareas_cantidad = df_inter_gobareas.groupby(['uta2020'])['fna'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gobareas_cantidad.rename(columns = {'fna':'reserva_cant'}, inplace=True)
# lista de nombres de áreas protegidas
df_inter_gobareas_nombres = df_inter_gobareas.loc[(df_inter_gobareas.fna.notnull()) & (df_inter_gobareas.fna != '-1')
    & (df_inter_gobareas.fna != '-2')& (df_inter_gobareas.fna != 'nan')].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gobareas_nombres.rename(columns = {'fna':'reserva_nombres'}, inplace=True)
# superficie por tipo de jurisdiccón
# jap: 2-nacional 3-pcial 4-muni 5-privado -1-universitaria
df_inter_gobareas_jap_sup = df_inter_gobareas.pivot_table(index="uta2020", columns="jap", values="sup_aprot_jap", aggfunc='sum').reset_index().fillna(0)
df_inter_gobareas_jap_sup.rename(columns = {-1.0:"reserva_jap_sindato", 1.0:"reserva_jap_univ", 2.0:"reserva_jap_nac", 3.0:"reserva_jap_pcial", 4.0:"reserva_jap_muni", 5.0:"reserva_jap_priv"}, inplace=True)
# superficie por tipo de reserva
# tap: 1-parque 2-reserva 3-monumento
df_inter_gobareas_tap_sup = df_inter_gobareas.pivot_table(index="uta2020", columns="tap", values="sup_aprot_jap", aggfunc='sum').reset_index().fillna(0)
df_inter_gobareas_tap_sup.rename(columns = {0.0:"reserva_tap_sindato", 1.0:"reserva_tap_parque", 2.0:"reserva_tap_reserva", 3.0:"reserva_tap_monum"}, inplace=True)

In [28]:
# generación de un campo booleano para el tipo y jurisdicción de las áreas protegidas 
# df_inter_gobareas_jap_sup['reserva_jap_univ_bool'] = 0
df_inter_gobareas_jap_sup['reserva_jap_nac_bool'] = 0
df_inter_gobareas_jap_sup['reserva_jap_pcial_bool'] = 0
df_inter_gobareas_jap_sup['reserva_jap_muni_bool'] = 0
df_inter_gobareas_jap_sup['reserva_jap_priv_bool'] = 0

# df_inter_gobareas_jap_sup.loc[df_inter_gobareas_jap_sup.reserva_jap_univ != 0, 'reserva_jap_univ_bool'] = 1
df_inter_gobareas_jap_sup.loc[df_inter_gobareas_jap_sup.reserva_jap_nac != 0, 'reserva_jap_nac_bool'] = 1
df_inter_gobareas_jap_sup.loc[df_inter_gobareas_jap_sup.reserva_jap_pcial != 0, 'reserva_jap_pcial_bool'] = 1
df_inter_gobareas_jap_sup.loc[df_inter_gobareas_jap_sup.reserva_jap_muni != 0, 'reserva_jap_muni_bool'] = 1
df_inter_gobareas_jap_sup.loc[df_inter_gobareas_jap_sup.reserva_jap_priv != 0, 'reserva_jap_priv_bool'] = 1

df_inter_gobareas_tap_sup['reserva_tap_parque_bool'] = 0
df_inter_gobareas_tap_sup['reserva_tap_reserva_bool'] = 0
df_inter_gobareas_tap_sup['reserva_tap_monum_bool'] = 0

df_inter_gobareas_tap_sup.loc[df_inter_gobareas_tap_sup.reserva_tap_parque != 0, 'reserva_tap_parque_bool'] = 1
df_inter_gobareas_tap_sup.loc[df_inter_gobareas_tap_sup.reserva_tap_reserva != 0, 'reserva_tap_reserva_bool'] = 1
df_inter_gobareas_tap_sup.loc[df_inter_gobareas_tap_sup.reserva_tap_monum != 0, 'reserva_tap_monum_bool'] = 1

In [29]:
# merge de todos los dataframes de áreas protegidas
dfs_areasreservas = [df_goblocales_aprot, df_inter_gobareas_cantidad, df_inter_gobareas_nombres, df_inter_gobareas_jap_sup, df_inter_gobareas_tap_sup]
dfs_areasreservas_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_areasreservas)
# nombres de columnas según estandarización
dfs_areasreservas_merged.rename(columns={'sup_aprot_k2':'k2_ap_sup', 'porc_sup_prot':'pc_ap_sup', 'reserva_cant':'u_ap_tot', 'reserva_nombres':'st_ap_nam',
    'reserva_jap_sindato':'k2_jur_sd', 'reserva_jap_nac':'k2_jur_nac', 'reserva_jap_pcial':'k2_jur_pro', 'reserva_jap_muni':'k2_jur_mun',
    'reserva_jap_priv':'k2_jur_pri', 'reserva_jap_nac_bool':'bo_jur_nac', 'reserva_jap_pcial_bool':'bo_jur_pro', 'reserva_jap_muni_bool':'bo_jur_mun',
    'reserva_jap_priv_bool':'bo_jur_pri', 'reserva_tap_sindato':'k2_tip_sd', 'reserva_tap_parque':'k2_tip_par', 'reserva_tap_reserva':'k2_tip_res',
    'reserva_tap_monum':'k2_tip_mon', 'reserva_tap_parque_bool':'bo_tip_par', 'reserva_tap_reserva_bool':'bo_tip_res', 'reserva_tap_monum_bool':'bo_tip_mon'}, inplace=True)
# formateo de los campos numéricos
dfs_areasreservas_merged.k2_ap_sup = round(dfs_areasreservas_merged.k2_ap_sup, 2)
dfs_areasreservas_merged.pc_ap_sup = dfs_areasreservas_merged.pc_ap_sup * 100
dfs_areasreservas_merged.pc_ap_sup = round(dfs_areasreservas_merged.pc_ap_sup, 2)
dfs_areasreservas_merged.k2_jur_sd = round(dfs_areasreservas_merged.k2_jur_sd, 2)
dfs_areasreservas_merged.k2_jur_nac = round(dfs_areasreservas_merged.k2_jur_nac, 2)
dfs_areasreservas_merged.k2_jur_pro = round(dfs_areasreservas_merged.k2_jur_pro, 2)
dfs_areasreservas_merged.k2_jur_pri = round(dfs_areasreservas_merged.k2_jur_pri, 2)
dfs_areasreservas_merged.k2_tip_sd = round(dfs_areasreservas_merged.k2_tip_sd, 2)
dfs_areasreservas_merged.k2_tip_par = round(dfs_areasreservas_merged.k2_tip_par, 2)
dfs_areasreservas_merged.k2_tip_res = round(dfs_areasreservas_merged.k2_tip_res, 2)
dfs_areasreservas_merged.k2_tip_mon = round(dfs_areasreservas_merged.k2_tip_mon, 2)
# extracción de los nombres de los campos
df_columns_areasreservas = list(dfs_areasreservas_merged.columns)

In [30]:
# cruce de los datos generados con los gobienros locales
df_goblocales_areasreservas = df_goblocales.merge(dfs_areasreservas_merged, how='left', on="uta2020")
df_goblocales_areasreservas[['k2_ap_sup', 'pc_ap_sup', 'u_ap_tot',
       'k2_jur_sd', 'k2_jur_nac', 'k2_jur_pro', 'k2_jur_mun', 'k2_jur_pri',
       'bo_jur_nac', 'bo_jur_pro', 'bo_jur_mun', 'bo_jur_pri', 'k2_tip_sd',
       'k2_tip_par', 'k2_tip_res', 'k2_tip_mon', 'bo_tip_par', 'bo_tip_res',
       'bo_tip_mon']] = df_goblocales_areasreservas[['k2_ap_sup', 'pc_ap_sup', 'u_ap_tot',
       'k2_jur_sd', 'k2_jur_nac', 'k2_jur_pro', 'k2_jur_mun', 'k2_jur_pri',
       'bo_jur_nac', 'bo_jur_pro', 'bo_jur_mun', 'bo_jur_pri', 'k2_tip_sd',
       'k2_tip_par', 'k2_tip_res', 'k2_tip_mon', 'bo_tip_par', 'bo_tip_res',
       'bo_tip_mon']].fillna(0)

# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_areasreservas = df_goblocales_areasreservas.drop(columns=['area_total_k2'])
df_goblocales_areasreservas.loc[df_goblocales_areasreservas['buffer'] == 1, ['st_ap_nam']] = None
df_goblocales_areasreservas.loc[df_goblocales_areasreservas['buffer'] == 1, ['k2_ap_sup', 'pc_ap_sup', 'u_ap_tot',
       'k2_jur_sd', 'k2_jur_nac', 'k2_jur_pro', 'k2_jur_mun', 'k2_jur_pri',
       'bo_jur_nac', 'bo_jur_pro', 'bo_jur_mun', 'bo_jur_pri', 'k2_tip_sd',
       'k2_tip_par', 'k2_tip_res', 'k2_tip_mon', 'bo_tip_par', 'bo_tip_res',
       'bo_tip_mon']] = float("NaN")

# exportación de los datos generados a csv y shp
df_goblocales_areasreservas[df_columns_areasreservas].to_csv("../BD_Procesada/MedioAmbienteReservasNaturales_GobiernosLocales.csv", encoding='utf-8')
schema_reservas_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 
                    'tipgobloc':'str', 'uta2020':'str',
                    'k2_ap_sup':'float:10.2', 'pc_ap_sup':'float:10.2', 'u_ap_tot':'int', 'st_ap_nam':'str', 'k2_jur_sd':'float:10.2', 
                    'k2_jur_nac':'float:10.2', 'k2_jur_pro':'float:10.2', 'k2_jur_mun':'float:10.2', 'k2_jur_pri':'float:10.2', 'bo_jur_nac':'int',
                    'bo_jur_pro':'int', 'bo_jur_mun':'int', 'bo_jur_pri':'int', 'k2_tip_sd':'float:10.2', 'k2_tip_par':'float:10.2',
                    'k2_tip_res':'float:10.2', 'k2_tip_mon':'float:10.2', 'bo_tip_par':'int', 'bo_tip_res':'int', 'bo_tip_mon':'int'}}
df_goblocales_areasreservas[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_areasreservas +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MedioAmbienteReservasNaturales_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_reservas_dic,
    encoding='utf-8')
df_goblocales_areasreservas.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,bo_jur_pro,bo_jur_mun,bo_jur_pri,k2_tip_sd,k2_tip_par,k2_tip_res,k2_tip_mon,bo_tip_par,bo_tip_res,bo_tip_mon
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,0.0,1.0,0.0,0.0,0.0,3.34,0.0,0.0,1.0,0.0
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,0.0,1.0,0.0,0.0,0.0,0.00,0.0,0.0,1.0,0.0
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


## Basurales

In [31]:
df_basurales.geometry = df_basurales.centroid
df_rellenos.geometry = df_rellenos.centroid

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [32]:
# cruce de las distintas capas con la capa de gob locales
df_goblocales_basurales = gpd.sjoin(df_basurales, df_goblocales, how="inner")
df_goblocales_rellenos = gpd.sjoin(df_rellenos, df_goblocales, how="inner")
df_goblocales_plseparacion = gpd.sjoin(df_plseparacion, df_goblocales, how="inner")
df_goblocales_plresiduos = gpd.sjoin(df_plresiduos, df_goblocales, how="inner")
df_goblocales_plefluentes = gpd.sjoin(df_plefluentes, df_goblocales, how="inner")
df_goblocales_plpotabil = gpd.sjoin(df_plpotabil, df_goblocales, how="inner")
# exportación de la capa resultante
df_goblocales_basurales.to_file("../CapasGeograficasCruzadas/Basurales_GobLocal.shp")
df_goblocales_rellenos.to_file("../CapasGeograficasCruzadas/Rellenos_GobLocal.shp")
df_goblocales_plseparacion.to_file("../CapasGeograficasCruzadas/PlantaSeparacion_GobLocal.shp")
df_goblocales_plresiduos.to_file("../CapasGeograficasCruzadas/PlantaResiduos_GobLocal.shp")
df_goblocales_plefluentes.to_file("../CapasGeograficasCruzadas/PlantaEfluentes_GobLocal.shp")
df_goblocales_plpotabil.to_file("../CapasGeograficasCruzadas/PlantaPotabilizadora_GobLocal.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  # Remove the CWD from sys.path while we load stuff.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if sys.path[0] == '':
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launche

In [33]:
# calculo de las variables
# cantidad de basurales
df_goblocales_basurales_count = pd.DataFrame(df_goblocales_basurales.value_counts("uta2020")).reset_index()
df_goblocales_basurales_count.columns = ['uta2020', 'basurales']
# cantidad de rellenos
df_goblocales_rellenos_count = pd.DataFrame(df_goblocales_rellenos.value_counts("uta2020")).reset_index()
df_goblocales_rellenos_count.columns = ['uta2020', 'rellenos']
# cantidad de plantas de separación
df_goblocales_plseparacion_count = pd.DataFrame(df_goblocales_plseparacion.value_counts("uta2020")).reset_index()
df_goblocales_plseparacion_count.columns = ['uta2020', 'planta_separ']
# cantidad de plantas de tratamiento de residuos
df_goblocales_plresiduos_count = pd.DataFrame(df_goblocales_plresiduos.value_counts("uta2020")).reset_index()
df_goblocales_plresiduos_count.columns = ['uta2020', 'planta_residuos']
# cantidad de plantas de tratamiento de efluentes
df_goblocales_plefluentes_count = pd.DataFrame(df_goblocales_plefluentes.value_counts("uta2020")).reset_index()
df_goblocales_plefluentes_count.columns = ['uta2020', 'planta_efluentes']
# cantidad de plantas de potabilizadoras de agua
df_goblocales_plpotabil_count = pd.DataFrame(df_goblocales_plpotabil.value_counts("uta2020")).reset_index()
df_goblocales_plpotabil_count.columns = ['uta2020', 'planta_potabl']

In [34]:
# merge de todos los dataframes de áreas protegidas
dfs_basurales = [df_goblocales_basurales_count, df_goblocales_rellenos_count, df_goblocales_plseparacion_count, df_goblocales_plresiduos_count, 
                df_goblocales_plefluentes_count, df_goblocales_plpotabil_count]
dfs_basurales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_basurales)
dfs_basurales_merged.fillna(0, inplace=True)

# generación del campo residuos_est, según el tipo de tratamiento
dfs_basurales_merged['residuos_est'] = 'Sin Tratar Ni Almacenar'
dfs_basurales_merged.loc[dfs_basurales_merged.basurales > 0, 'residuos_est'] = 'Basural Sin Tratamiento'
dfs_basurales_merged.loc[(dfs_basurales_merged.basurales > 0) & ((dfs_basurales_merged.rellenos > 0) | (dfs_basurales_merged.planta_separ > 0)), 'residuos_est'] = 'Basural Con Tratamiento'
dfs_basurales_merged.loc[(dfs_basurales_merged.basurales == 0) & ((dfs_basurales_merged.rellenos > 0) | (dfs_basurales_merged.planta_separ > 0)), 'residuos_est'] = 'Con Tratamiento'
# nombres de columnas según estandarización
dfs_basurales_merged.rename(columns={'basurales':'u_bas_tot', 'rellenos':'u_rel_tot', 'planta_separ':'u_pse_tot', 'planta_residuos':'u_pre_tot', 'planta_efluentes':'u_pef_tot', 
'planta_potabl':'u_ppo_tot', 'residuos_est':'st_tip'}, inplace=True)
# extracción de los nombres de los campos
df_columns_basurales = list(dfs_basurales_merged.columns)

In [35]:
# cruce de los datos generados con los gobienros locales
df_goblocales_basural_gen = df_goblocales.merge(dfs_basurales_merged, how='left', on="uta2020")
df_goblocales_basural_gen.u_bas_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.u_rel_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.u_pse_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.u_pre_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.u_pef_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.u_ppo_tot.fillna(0, inplace=True)
df_goblocales_basural_gen.st_tip.fillna('Sin Tratar Ni Almacenar', inplace=True)

# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_basural_gen = df_goblocales_basural_gen.drop(columns=['area_total_k2'])
df_goblocales_basural_gen.loc[df_goblocales_basural_gen['buffer'] == 1, ['st_tip']] = None
df_goblocales_basural_gen.loc[df_goblocales_basural_gen['buffer'] == 1, ['u_bas_tot', 'u_rel_tot', 'u_pse_tot',
       'u_pre_tot', 'u_pef_tot', 'u_ppo_tot']] = float("NaN")

# exportación de los datos generados a csv y shp
df_goblocales_basural_gen[df_columns_basurales].to_csv("../BD_Procesada/MedioAmbienteBasuralesPlantas_GobiernosLocales.csv", encoding='utf-8')
schema_basural_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 
                    'tipgobloc':'str', 'uta2020':'str',
                    'u_bas_tot':'int', 'u_rel_tot':'int', 'u_pse_tot':'int', 'u_pre_tot':'int', 'u_pef_tot':'int', 
                    'u_ppo_tot':'int', 'st_tip':'str'}}
df_goblocales_basural_gen[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_basurales +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MedioAmbienteBasuralesPlantas_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_basural_dic,
    encoding='utf-8')
df_goblocales_basural_gen.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,Longitud,buffer,geometry,u_bas_tot,u_rel_tot,u_pse_tot,u_pre_tot,u_pef_tot,u_ppo_tot,st_tip
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,-58.37153965302685,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",0.0,0.0,0.0,1.0,0.0,0.0,Sin Tratar Ni Almacenar
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,-58.39489181181946,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",0.0,0.0,0.0,0.0,0.0,0.0,Sin Tratar Ni Almacenar
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,-58.40265145122171,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",0.0,0.0,0.0,0.0,0.0,0.0,Sin Tratar Ni Almacenar
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,-58.38745506898847,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",0.0,0.0,0.0,6.0,0.0,0.0,Sin Tratar Ni Almacenar
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,-58.42057219028574,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",0.0,0.0,0.0,0.0,0.0,0.0,Sin Tratar Ni Almacenar
